
# Movie Review Sentiment Analysis
This Jupyter Notebook details the implementation of a binary linear classifier for sentiment analysis of movie reviews.
The classifier is developed using Python without external libraries like numpy or Scikit-learn.


In [8]:
import requests

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(response.content)

util_url = 'https://raw.githubusercontent.com/Rezaabdi78/AdvAI-SentimentAnalysis-P3/main/util.py'
train_data_url = 'https://raw.githubusercontent.com/Rezaabdi78/AdvAI-SentimentAnalysis-P3/main/polarity.train'
validation_data_url = 'https://raw.githubusercontent.com/Rezaabdi78/AdvAI-SentimentAnalysis-P3/main/polarity.dev'
grader_url = 'https://raw.githubusercontent.com/Rezaabdi78/AdvAI-SentimentAnalysis-P3/main/grader.py'
graderUtil_url = 'https://raw.githubusercontent.com/Rezaabdi78/AdvAI-SentimentAnalysis-P3/main/graderUtil.py'
submission_url = 'https://raw.githubusercontent.com/Rezaabdi78/AdvAI-SentimentAnalysis-P3/main/submission.py'

download_file(util_url, 'util.py')
download_file(train_data_url, 'polarity.train')
download_file(validation_data_url, 'polarity.dev')
download_file(grader_url, 'grader.py')
download_file(graderUtil_url, 'graderUtil.py')
download_file(submission_url, 'submission.py')


import random
from typing import Callable, Dict, List, Tuple, TypeVar

from util import *

FeatureVector = Dict[str, int]
WeightVector = Dict[str, float]
Example = Tuple[FeatureVector, int]


## Problem 3a: Feature Extraction
**Objective:** Implement the `extractWordFeatures` function to convert a movie review (string) into a feature vector.
Each word in the string is treated as a feature, and its frequency in the string is its corresponding value in the feature vector.


In [2]:

def extractWordFeatures(x: str) -> dict:
    words_list = x.split(' ')
    occurance_counter = dict()
    for word in words_list:
        occurance_counter[word] = occurance_counter.get(word, 0) + 1
    return occurance_counter



## Problem 3b: Stochastic Gradient Descent (SGD)
**Objective:** Implement the `learnPredictor` function using stochastic gradient descent to minimize hinge loss for binary classification. This function iteratively updates a weight vector based on the training examples, aiming to reduce the classification error.

**Methodology:**
- SGD iteratively processes each training example and updates the model's weights.
- Hinge loss is used as the loss function, which is suitable for binary classification tasks.
- The function tracks training and validation errors to monitor the learning process.


In [3]:

def learnPredictor(trainExamples, validationExamples, featureExtractor, numEpochs, eta):
    weights = {}  # feature => weight

    def predictor(x):
        features = featureExtractor(x)
        return 1 if dotProduct(features, weights) >= 0 else -1

    for epoch in range(numEpochs):
        for x, y in trainExamples:
            extractedFeatures = featureExtractor(x)
            if y * dotProduct(extractedFeatures, weights) < 1:
                gradient = {feature: -y * value for feature, value in extractedFeatures.items()}
                increment(weights, -eta, gradient)

        trainError = evaluatePredictor(trainExamples, predictor)
        validationError = evaluatePredictor(validationExamples, predictor)
        print(f"Epoch {epoch + 1}: Training error = {trainError}, Validation error = {validationError}")

    return weights



## Problem 3c: Generate Test Case
**Objective:** Develope the `generateDataset` function to create synthetic examples that are correctly classified by a given weight vector. This function aids in verifying the effectiveness of the classifier.

**Methodology:**
- The function generates data points that align with the classifier's decision boundary.
- The synthetic data should be representative of the types of examples the classifier will encounter.
- This approach is useful for testing and debugging the classifier.


In [4]:

def generateDataset(numExamples: int, weights: WeightVector) -> List[Example]:
    random.seed(42)


    def generateExample() -> Tuple[Dict[str, int], int]:
        # BEGIN_YOUR_CODE (our solution is 3 lines of code, but don't worry if you deviate from this)
        random_length = random.randint(0, len(weights))
        phi = dict()
        for i in range(random_length):
            phi.update({key: value for key, value in random.choice([weights.items()])})
        random_y = int(random.uniform(-2, 2))
        y = random_y if random_y not in [0, -1] else 1
        # END_YOUR_CODE
        return phi, y

    return [generateExample() for _ in range(numExamples)]



## Problem 3d: Character Features
**Objective:** Implement the `extractCharacterFeatures` function to extract character n-gram features from a string. This approach considers sequences of characters, ignoring whitespace, to form features.

**Methodology:**
- Character n-grams provide a way to capture textual patterns at a finer granularity than whole words.
- This method can be particularly effective for languages where word boundaries are less clear.
- The function should count occurrences of each n-gram in the input string.


In [5]:

def extractCharacterFeatures(n: int) -> Callable[[str], FeatureVector]:
    def extract(x: str) -> Dict[str, int]:
        # Example implementation (to be replaced with actual logic)
        result = {}
        x = x.replace(" ", "")  # Remove whitespace
        for i in range(len(x) - n + 1):
            ngram = x[i:i+n]
            result[ngram] = result.get(ngram, 0) + 1
        return result
    return extract



## Problem 3e: Testing Different Values of N
**Objective:** Explore the impact of different values of 'n' in the `extractCharacterFeatures` function to determine the optimal length of n-grams that minimizes validation error.

**Methodology:**
- Testing different 'n' values helps in understanding how the granularity of n-grams affects model performance.
- The goal is to find a balance where the n-grams are informative enough to aid classification but not so detailed as to overfit or become too sparse.


In [6]:

def testValuesOfN():
    # Example implementation (to be replaced with actual logic)
    for n in range(1, 5):  # Example range of n
        featureExtractor = extractCharacterFeatures(n)
        # Code to train model and evaluate performance
        print(f"Testing with n = {n}, Performance: ...")


In [9]:
# Run grader.py script
!python grader.py


## Problem 3e: Experimenting with Different Values of N

### Experimental Setup
Run the linear predictor with the `extractCharacterFeatures` feature extractor for different values of `n`. We aim to find the value of `n` that produces the smallest validation error.


In [13]:
trainExamples = readExamples('polarity.train')
validationExamples = readExamples('polarity.dev')
numEpochs = 20
eta = 0.01


def run_character_feature_experiment(trainExamples, validationExamples, n_values):
    for n in n_values:
        featureExtractor = extractCharacterFeatures(n)
        weights = learnPredictor(trainExamples, validationExamples, featureExtractor, numEpochs, eta)
        validationError = evaluatePredictor(validationExamples, lambda x: (1 if dotProduct(featureExtractor(x), weights) >= 0 else -1))
        print(f'n = {n}: Validation Error = {validationError}')

n_values = [1, 2, 3, 4, 5]  # Example range of n
run_character_feature_experiment(trainExamples, validationExamples, n_values)

Read 3554 examples from polarity.train
Read 3554 examples from polarity.dev
Epoch 1: Training error = 0.47326955543050087, Validation error = 0.4935284186831739
Epoch 2: Training error = 0.449634214969049, Validation error = 0.4766460326392797
Epoch 3: Training error = 0.44034890264490717, Validation error = 0.467923466516601
Epoch 4: Training error = 0.4296567248171075, Validation error = 0.4631401238041643
Epoch 5: Training error = 0.4569499155880698, Validation error = 0.48086662915025324
Epoch 6: Training error = 0.4662352279122116, Validation error = 0.4879009566685425
Epoch 7: Training error = 0.44991558806978055, Validation error = 0.475801913337085
Epoch 8: Training error = 0.4752391671356218, Validation error = 0.4926842993809792
Epoch 9: Training error = 0.4651097355092853, Validation error = 0.48480585256049524
Epoch 10: Training error = 0.4448508722566123, Validation error = 0.4583567810917276
Epoch 11: Training error = 0.4617332583005065, Validation error = 0.4887450759707

### Observations and Analysis
Through the experiment, we observed that the smallest validation error was achieved with n=5. This suggests that character n-grams of this length capture essential features of the text effectively, balancing specificity and generalization. Such n-grams can encapsulate partial words or word segments, which might be more robust to variations in word forms than whole words.

### Practical Example
Consider the review: "Unbelievably thrilling!" In this case, character n-grams might outperform word features as they can capture the essence of compounded words like 'unbelievably'. These n-grams can detect patterns in character sequences that transcend traditional word boundaries, making them more effective for such text.

## Conclusion

This notebook presented a detailed exploration of building a binary linear classifier for sentiment analysis of movie reviews. Key highlights and learnings are:

- **Feature Extraction (Problem 3a):** We implemented a method to extract word features from text, demonstrating the importance of feature representation in machine learning models.
- **Stochastic Gradient Descent (Problem 3b):** The application of SGD for hinge loss minimization highlighted the iterative nature of optimizing classification models.
- **Synthetic Test Cases (Problem 3c):** Generating synthetic data provided insights into model behavior and helped in model validation.
- **Character Features (Problem 3d and 3e):** Experimentation with character n-grams revealed their effectiveness in capturing linguistic patterns, especially in scenarios where traditional word boundaries are less distinct.

Overall, this exercise emphasized the significance of feature selection, the impact of algorithmic choices in model performance, and the value of rigorous testing. These components are crucial in developing robust machine learning models for natural language processing tasks.
